In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langgraph.graph import StateGraph, END
from typing import Dict, Any


# Define prompt templates
nurse_prompt = ChatPromptTemplate.from_template(
    "Role: ER Triage Nurse\nInstructions: Determine ESI (1-5) with reasoning...\nPatient Note: {note}\nPrevious Doctor Input: {doctor_msg}\nYour Response:"
)
doctor_prompt = ChatPromptTemplate.from_template(
    "Role: ER Doctor\nInstructions: List possible diagnoses and comment on priority...\nPatient Note: {note}\nNurse's Assessment: {nurse_msg}\nYour Response:"
)

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

from langgraph.graph import StateGraph, START, END 
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
# llm = ChatOpenAI(
#     model="gpt-4o-mini",
#     temperature=0)

In [3]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")


In [4]:
def nurse_step(state: Dict[str, Any]) -> Dict[str, Any]:
    """Process nurse's assessment with state preservation"""
    inputs = {
        "note": state["note"],
        "doctor_msg": state.get("doctor_msg", "")
    }
    response = (nurse_prompt | llm).invoke(inputs)
    return {**state, "nurse_msg": response.content}  # Preserve existing state

def doctor_step(state: Dict[str, Any]) -> Dict[str, Any]:
    """Process doctor's analysis with state preservation"""
    inputs = {
        "note": state["note"],
        "nurse_msg": state["nurse_msg"]
    }
    response = (doctor_prompt | llm).invoke(inputs)
    return {
        **state,  # Preserve existing state
        "doctor_msg": response.content,
        "iteration": state.get("iteration", 0) + 1
    }

# Build workflow
workflow = StateGraph(state_schema=dict)
workflow.add_node("Nurse", nurse_step)
workflow.add_node("Doctor", doctor_step)
workflow.set_entry_point("Nurse")
workflow.add_edge("Nurse", "Doctor")
workflow.add_conditional_edges(
    "Doctor",
    lambda state: "Nurse" if state.get("iteration", 0) < 2 else END
)

app = workflow.compile()

# Test run
patient_note = "45yo male presenting with chest pain radiating to left arm..."
result = app.invoke({"note": patient_note})

print("Final Assessment:", result["nurse_msg"])
print("\nFinal Analysis:", result["doctor_msg"])

Final Assessment: **ESI: 2**

**Reasoning:**

This patient presents with a high-risk chief complaint (chest pain radiating to the left arm) suggestive of a potentially life-threatening condition like Acute Coronary Syndrome (ACS).

*   **High-Risk Situation:** Chest pain, especially with radiation, requires immediate assessment to rule out cardiac ischemia or other critical conditions.
*   **Likely Need for Immediate Intervention:** This patient will likely require interventions such as ECG, oxygen, IV access, cardiac monitoring, and potentially medications (aspirin, nitroglycerin, morphine).
*   **Not ESI 1:** While the presentation is concerning, the patient's current stability is unknown. If the patient was hypotensive, unresponsive, or in severe respiratory distress, ESI 1 would be more appropriate.
*   **Not ESI 3, 4, or 5:** The high-risk nature of the chief complaint and the likely need for multiple resources and interventions within a defined period (minutes to hours) rules out

In [5]:
# Add validation checks
assert "note" in result, "Patient note missing in final state"
assert result["iteration"] == 2, f"Unexpected iterations: {result['iteration']}"
assert "chest pain" in result["nurse_msg"], "Key symptom missing in nurse analysis"
assert "Acute Coronary Syndrome" in result["doctor_msg"], "Key diagnosis missing"

In [6]:
critical_note = "30yo female unresponsive with BP 70/40..."
critical_result = app.invoke({"note": critical_note})

In [27]:
critical_result["nurse_msg"]

'Based on the provided information, the patient is a 30-year-old female who is unresponsive and has a blood pressure of 70/40 mmHg. This presentation is concerning for a life-threatening condition, and immediate intervention is required.\n\n**Emergency Severity Index (ESI) Level: 1**\n\n**Reasoning:**\n- **Unresponsiveness**: The patient is unresponsive, indicating a potential compromise in the airway, breathing, or circulation. This is a critical sign that necessitates immediate medical attention.\n- **Hypotension (BP 70/40)**: The extremely low blood pressure suggests a state of shock, which can be due to several life-threatening conditions such as septic shock, hypovolemic shock, or cardiogenic shock. Each of these conditions requires urgent intervention to prevent further deterioration and potential death.\n- **Need for Immediate Intervention**: The patient requires rapid assessment and treatment, including airway management, fluid resuscitation, and possibly medications (e.g., ant